In [ ]:
# Cargar los datos de entrenamiento

import numpy as np

train_matrix = np.load('../features/matriz_datos_train.npy')

# Selecionar las features y target, es decir, separar los Atributos de la clase
# donde la matriz X será la matríz de Atributos y el vector Y será el vector de la clase

# Atributos
X_train = train_matrix[:, :-1]

# Clase
y_train = train_matrix[:, -1]

# Eliminaremos atributos que no contribuyen a nuestro modelo y solamente generan ruido

# Dependiendo del modelo que seguimos, podemos aplicar antes o después la estandarización
# En clase vimos el método del variance Threshold que lo que hace es eliminar los atributos que 
# no tienen una varianza muy grande (dependiendo del parametro que nosotros pongamos)

# Sin embargo, para este metodo lo que haremos primero será aplicar la estandarización
# Ya estandarizados analizaremos la distribución que siguen esas variables
# y en base a la distribución aplicaremos una selección de atributos u otra

In [ ]:
# Estandarización de los datos de entrenamiento. Los de test de momento los dejamos de lado

from sklearn.preprocessing import StandardScaler

estandarizador = StandardScaler()

# Ya entrenado el objeto estandarizador, puedo extraer las variables que son importantes en esta funcion
estandarizador.fit(X_train)

# Lo que hace el metodo StandardScaler es coger cada uno de los valores para una columna
# y le resta a cada valor de la columna la media de dicha columna y lo divide entre la desviación
# típica de esa columna

# Hace un centrado y un escalado en base a la media y la desviación estandar

mu = estandarizador.mean_
sigma = np.sqrt(estandarizador.var_)

# Conjunto de datos ya estandarizado
X_train = estandarizador.transform(X_train)

In [ ]:
# Selección de los atributos o características

# Primero estudiaremos la normalidad de los datos para saber si siguen una distribución normal
# Si siguen una distribución normal haremos una comparación de media
# y en caso contrario haremos una comparación de la mediana

# Estudiar si las variables siguen una distribución normal de media 0 y desviación típica 1 ---> N(0, 1)
# Kstest hace referencia a la prueba de Kolmogorov-Smirnov
from scipy.stats import kstest

# Nivel de confianza del 99%
alpha = 0.01
h_norm = np.zeros(X_train.shape[1])

for i in range(0, X_train.shape[1]):
     _, pvalue = kstest(X_train[:, i], 'norm')
    
    # Contraste de hipotesis
    id pvalue <= aplha:
        # Los datos NO siguen una distribución normal N(0, 1)
        h_norm[i] = 0
    
    else:
        # Los datos Si siguen una distribución normal N(0, 1)
        h_norm[i] = 1

In [ ]:
import matplotlib.pyplot as plt

def draw_boxplot(data_1, data_2, ticks):
    
    bp1 = plt.boxplot(data_1, positions=np.array(range(np.shape(data_1)[1]))*2.0-0.4, sym='', widths=0.5, \
                     boxprops=dict(color='red'),
                     capprops=dict(color='red'),
                     whiskerprops=dict(color='red'),
                     medianprops=dict(colot='red'))
    
    bp1 = plt.boxplot(data_2, positions=np.array(range(np.shape(data_2)[1]))*2.0+0.4, sym='', widths=0.5, \
                     boxprops=dict(color='blue'),
                     capprops=dict(color='blue'),
                     whiskerprops=dict(color='blue'),
                     medianprops=dict(colot='blue'))
                      
    plt.plot([], c='#D7191C', label='Glaucoma')
    plt.plot([], c='#2C7DB6', label='Healthy')
    plt.legend()
    
    plt.xticks(range(0, len(ticks)*2, 2), ticks)
    plt.xlim(-2, len(ticks)*2)
    plt.grid(True)
    plt.title('Características')
    plt.show()s

In [ ]:
# Estudiar la capacidad discriminativa de los atributos en función de su distribución
# Con la ttest_ind hago una comparación de medias y con mannhwhitneyu hago una comparación de medianas 
from scipy.stats import ttest_ind, mannhwhitneyu

glaucoma_data = X_train[y_train == 1]
healthy_data = X_train[y_train == 0]

h = np.zeros(X_train.sahep[1])
h_disc = np.zeros(X_train.shape[1])

for i in range(0, X_train.shape[1]):
    # Si no es normal --> comparación de medianas
    if h_norm[i] == 0:
        # Lo que estoy haciendo aqui es comparar las medianas de los valores que toma la característica i
        # cuando la muestra es de glaucoma y cuando la muestra es sana
        
        # A nosotros nos interesa que en este punto las medianas sean diferentes por que eso me indicara que
        # hay una dependencia entre la característica y la clase
        # Digamos que nos interesa que tome valores diferentes de mediana cuando la clase es de glaucoma
        # que cuando la clase es sana ya que esto ayudara en la fase de modelado
        # al modelo para distinguir entre una muestra sana y una con glaucoma
        _, pvalue = mannhwhitneyu(glaucoma_data[:,i], healthy_data[:,i])
        
        # Si obtenemos pvalue con valores de mediana iguales, discriminamos ese atributo.
    # Si es normal --> comparación de medias
    else:
        # De igual manera discriminaremos aquellas que la media sea la misma
        _, pvalue = ttest_ind(glaucoma_data[:,i], healthy_data[:,i])
        
    # Contraste de hipotesis, estudiar el poder discriminatorio de las características
    # Definimos la hipotesis nula
    # H0: Independencia entre la característica y la clase
    if pvalue <= aplha:
        # Se rechaza la hipotesis nula, y por tanto, asumimos la dependencia entre 
        # la característica y la clase
        h_disc[i] = 1
    else:
        # No hay evidencia para rechazar la HO, y por tanto, asumimos que la caract. y la clase son independientes
        # Tenemos que eliminar estos atributos ya que son independientes de la clase y no aportan
        h_disc[i] = 0

# Eliminando las variables que no son discriminatorias ya que si las mantengo solo meto ruido al modelo
# afectando el rendimiento de nuestro modelo

id_no_disc = np.where(h_disc == 0)

# Eliminamos de nuestra matriz de datos original los atributos que no discriminan 
X_train_disc = np.delete(X_train, id_no_disc[0], axis=1)

mu_disc = np.delete(mu, id_no_disc[0])
sigma_disc = np.delete(sigma, id_no_disc[0])

# Visualización para enfrentar el diagrama de caja y bigotes
original_ticks = ['media', 'mediana', 'std', 'asim', 'curtosis', 'min', 'max', 'con', 'dis', 'homo', 'ASM',
                  'E', 'COR', 'LBP1', 'LBP2', 'LBP3', 'LBP4', 'LBP5', 'LBP6', 'LBP7', 'LBP8', 'LBP9', 'LBP10']

draw_boxplot(glaucoma_data, healthy_data, original_ticks)

# Eliminamos características que han resultado no ser discriminantes
ticks = np.delete(original_ticks, id_no_disc[0])

In [ ]:
# Una vez que hemos obtenidos que variables tienen capacidad discriminativa 
# y cuales no, ahora lo que haremos será estudiar de manera cuantitativa que variables están 
# correladas con otras ya que aportan la misma información al modelo.
# Haremos esto para obtener un modelo más robusto en términos de coste computacional

# Realizar un análisis de CORRELACIÓN para ver la dependencia entre pares de variables
# En este punto nos interesa la INDEPENDENCIA entre variables, ya que queremos eliminar aquellas
# variables que aportan la misma información al modelo

import matplotlib.pyplot as plt

# Matriz de correlación de variables discriminantes
R = np.corrcoef(X_train_disc.transpose())

# Esta matriz nos permite observar de manera visual la correlación entre diferentes pares de variables
# Mientras más rojo sea el color, quiere decir que hay una correlación más fuerte
plt.imshow(R, cmap='jet')
plt.show()

# Umbral de correlación. Si 2 variables están correlacionadas más de un 90%
# entonces podemos pasar a descartar esa variable del estudio
th_cor = 0.9

# Vamos a declarar un indice en valor absoluto en donde
# indicaremos en que lugar ese indice es mayor que el umbral
# Lo ponemos en valor absoluto ya que no nos importa si hay una correlación positiva o negativa
# solo nos importa si están correladas o no
idx = abs(R) > th_cor
mat_tri_sup = np.triu(idx, 1)

row, cow = np.where(mat_tri_sup == True)
id_corr = np.unique(col)

# Eliminamos las variables correlacionadas
X_final = np.delete(X_train_disc, id_corr, axis=1)
mu_final = np.delete(mu_disc, id_corr)
sigma_final = np.delete(sigma_disc, id_corr)
ticks = np.delete(ticks, id_corr)

In [ ]:
# Guardado de matriz final de características
import os

if not os.path.exists('../final_features'):
    os.mkdir('../final_features')
    
y_train_exp = np.expand_dims(y_train, axis=1)
train_matrix = np.concatenate((X_fianl, y_train_exp), axis=1)
np.save('../final_features/train.npy', train_matrix)

In [ ]:
# Ahora prepararemos nuestro set de datos test
# repetiremos el proceso para la selección de las características del test

test_matrix = np.load('../features/matriz_datos_test.npy')

# Seleccionar las características y la clase

X_test = test_matrix[:,:-1]
y_test = test_matrix[:, -1]

# Eliminar las características que no son discriminatorias del conjunto test durante entrenamiento
X_test_disc = np.delete(X_test, id_not_disc[0], axis=1)

# Ahora eliminamos las características que están correladas durante entrenamiento
X_test_fianl = np.delete(X_test_disc, id_corr, axis=1)

# Ahora tenemos que estandarizar nuestro set de test en base a la mu y la sigma del entrenamiento
# Esto sería al equivalente del .transform del objeto estandarizado con el set de train
# sin embargo ya que hemos realizado la estandarización antes de la selección de atributos
# hemos tenido que ir arrastrando la mu y la sigma para poder hacer luego el centrado y el escalado
X_test_final = (X_test_final-mu_final)/sigma_final

y_test_exp = np.expand_dims(y_test, axis=1)
test_matrix = np.concatenate((X_test_final, y_test_exp), axis=1)

np.save('../final_features/test.npy', test_matrix)

# De esta manera queda concluida la fase de selección de características y podemos pasar a la fase de modelado